# 実験実行環境を再構築した場合のセットアップを行う

実験パッケージのREADME.mdから、実験実行環境の再構築した場合に必要な初期セットアップを行います。  
このタスクを実行すると、実験データの更新や実験フローの利用が可能になります。  
上から順番に実行してください。

2回目以降の実行の場合は、このセルが選択された状態で画面上部に表示される以下のボタンをクリックしてセルの解凍を行ってから、セルを実行して下さい。  
![UnfreezeBotton](../../images/unfreeze_button.png)

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('blue')

## 1. ユーザー認証を行う

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行し、画面の表示に沿ってGIN-forkに登録したユーザー名、パスワード、メールアドレスを入力してください。各項目入力後、Enterキーを押下してください。  

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

tokens = []
access_token = {}

# 正常に認証が終わるまで繰り返し
path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils
tokens, access_token, name, email = utils.verify_GIN_user()

os.chdir(os.environ['HOME'])
!git config --global user.name $name
!git config --global user.email $email
clear_output()
print("認証が正常に完了しました。次の手順へお進みください。")

## 2. データ同期のための設定をする

この手順では、今の実行環境とGIN-forkのリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json

os.chdir(path_flow_root)
from util.scripts import utils

params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)
pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'/api/v1/user/keys?token=' + tokens[0]['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

if response.status_code == HTTPStatus.CREATED:
    print('公開鍵の用意が出来ました。')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('すでに公開鍵の用意が完了しています。')

In [ ]:
import os
import subprocess
from datalad import api
from IPython.display import clear_output

# sibling url をsshに変更する
os.chdir(os.environ['HOME'])
http_url = subprocess.getoutput('git config --get remote.origin.url')
ssh_url = http_url.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])
    
# siblingsにGINを登録する
sibling = !datalad siblings -s gin
for item in sibling:
    if 'unknown sibling name' in item:
        api.siblings(action='add', name='gin', url=ssh_url)
    else:
        pass

clear_output()
print('SSH接続の準備が完了しました。')

In [ ]:
import os
os.chdir(path_flow_root)
from util.scripts import utils

# SSHホスト（＝GIN）を信頼する設定
utils.config_GIN(ginHttp = params['siblings']['ginHttp'])

# S3にあるデータをGIN-forkに同期しないための設定
!git annex untrust here
!git annex --force trust web

## 3. 当実行環境で編集する実験パッケージを選択する

以下のセルを実行して、この実験実行環境で編集したい実験パッケージを1つ選択してください。  
選択を間違えた場合は、再度このセルから実行して、選択しなおしてください。

In [ ]:
import json
from ipywidgets import Dropdown, Button, Layout
from IPython.display import clear_output

with open('/home/jovyan/pipeline.json', 'r') as f:
    pipeline = json.load(f)

def on_click_callback(clicked_button: Button) -> None:
    global experiment_title
    experiment_title=dropdown.value
    clear_output()
    print("入力を受けつけました：", experiment_title)

dropdown = Dropdown(
    options=pipeline,
    description='実験パッケージ名:',
    disabled=False,
    style= {'description_width': 'initial'}
)

button = Button(description='入力完了')
button.on_click(on_click_callback)
print("実験パッケージの選択後、入力完了ボタンを押下してください。")
display(dropdown, button)

## 4. 実験パッケージのデータを用意する

当実験パッケージのデータをこの実験実行環境に用意します。

In [ ]:
import os
from IPython.display import clear_output
import logging

EXPERIMENT_TITLE = experiment_title
%store EXPERIMENT_TITLE
clear_output()
os.chdir(os.environ['HOME'] + '/experiments/' + experiment_title)
try:
    !datalad get .
    !datalad unlock .
except Exception as e:
    logging.error('データの用意に失敗しました。')
    logging.error(str(e))
else:
    clear_output()
    print('データの用意が完了しました。次にお進みください。')

## 5. 本タスクの実行結果をGIN-forkに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](https://dg.nii.ac.jp/G-Node/Info/wiki/%E3%83%AA%E3%82%B5%E3%83%BC%E3%83%81%E3%83%95%E3%83%AD%E3%83%BC%E6%A9%9F%E8%83%BD#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

%store -r EXPERIMENT_TITLE
git_path = '/home/jovyan/WORKFLOWS/EX-WORKFLOWS/util/required_rebuild_container.ipynb'
is_ok = utils.syncs_with_repo(git_path=git_path, gitannex_path=None, gitannex_files=None, message=EXPERIMENT_TITLE + '_実験環境再構築用セットアップ')

## 6. フロー図を更新する
フロー図にこのタスクが実行済みであることを反映します。

In [ ]:
import os
os.chdir(os.environ['HOME'])

path = 'WORKFLOWS/EX-WORKFLOWS/images/notebooks.diag'

with open('.gitignore', 'r') as f:
    text = f.read()
    if text.find(path) == -1:
        !echo $path >> ./.gitignore

# notebooks.diagのgit管理を外す
!git update-index --skip-worktree $path

# フロー図に不要マークをつける
find = '"required_every_time"[fontsize = 10];'
replace = '"required_every_time"[numbered = 不要, fontsize = 10];'

with open(path, 'r') as f:
    s = f.read()

with open(path, 'w') as f:
    s = s.replace(find, replace)
    f.write(s)

expath = 'WORKFLOWS/images/notebooks.diag'
with open('.gitignore', 'r') as f:
    text = f.read()
    if text.find(expath) == -1:
        !echo $expath >> ./.gitignore
!git update-index --skip-worktree $expath

## 7. 実験フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックしてください。実験フロートップページに遷移し、実験フローをご利用いただけます。  
実験フロートップページのフロー図の「初期設定を行う」は実行いただく必要はありません。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../experiment.ipynb'>実験フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))